In [1]:
import csv
import cv2
import numpy as np
import os
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

lines = []

# Read each line of csv file and store it in lines array
def csv_reader(filename, lines_in):
    with open(filename) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            lines_in.append(line)
    return lines_in
lines = csv_reader('../track1/laps.csv', lines)
lines = csv_reader('../track1/backward.csv', lines)
lines = csv_reader('../track1/backward.csv', lines)
lines = csv_reader('../track1/recovery.csv', lines)
print('Total images/camera captured = ', len(lines))

# Split data in training and validation samples
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

# Generator function to load data and preprocess in batchsize portions 
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            # Read image from path given in the CSV file and store it in images array
            # Store steering measurements in measurements array
            images = []
            measurements = []
            
            for batch_sample in batch_samples:
                for i in range(3):
                    source_path = batch_sample[i] #Index 0:center 1:left 2:right
                    filename = source_path.split('/')[-1]
                    current_path = '../track1/IMG/' + filename
                    image = cv2.imread(current_path)
                    images.append(image)

                    # Steering measurement + correction based on center/left/right image
                    if i == '0':
                        measurement = float(batch_sample[3])
                    elif i == '1':
                        measurement = float(batch_sample[3]) + 0.2
                    else: # i == '2'
                        measurement = float(batch_sample[3]) - 0.2        
                    measurements.append(measurement)

                    # Augmenting data by flipping images
                    images.append(cv2.flip(image,1))
                    measurements.append(measurement*-1.0)

            # Images are features and steering measurements are labels
            # Convert to numpy arrays for keras modeling
            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

# Keras model (Neural Network)
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Dense, Flatten, Lambda, Dropout, Cropping2D
model = Sequential()

# Normalize x/255 and subract by 0.5 for zero mean
model.add(Lambda(lambda x: x/255 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (60,60))))

# Nvidia architecture
# Layer1: Input = 65x200x3 Output = 31x98x24
model.add(Convolution2D(24,5,5,activation="relu"))
model.add(MaxPooling2D())
# Layer2: Output = 14x47x36
model.add(Convolution2D(36,5,5,activation="relu"))
model.add(MaxPooling2D())
# Layer3: Output = 5x22x48
model.add(Convolution2D(48,5,5,activation="relu"))
model.add(MaxPooling2D())
#Layer4: Output = 2x10x64
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, \
                    samples_per_epoch=len(train_samples), \
                    validation_data=validation_generator, \
                    nb_val_samples=len(validation_samples), \
                    nb_epoch=15)

model.save('model.h5')

Total images/camera captured =  3731


Using TensorFlow backend.


Epoch 1/15
2880/2984 [===========================>..] - ETA: 0s - loss: 0.1278

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


3072/2984 [==============================] - 11s - loss: 0.1270 - val_loss: 0.1178
Epoch 2/15
3072/2984 [==============================] - 8s - loss: 0.0910 - val_loss: 0.0852
Epoch 3/15
3072/2984 [==============================] - 8s - loss: 0.0870 - val_loss: 0.0696
Epoch 4/15
3072/2984 [==============================] - 9s - loss: 0.0763 - val_loss: 0.1095
Epoch 5/15
3072/2984 [==============================] - 8s - loss: 0.0909 - val_loss: 0.0739
Epoch 6/15
3120/2984 [===============================] - 9s - loss: 0.0746 - val_loss: 0.0764
Epoch 7/15
3072/2984 [==============================] - 8s - loss: 0.0839 - val_loss: 0.0802
Epoch 8/15
3072/2984 [==============================] - 8s - loss: 0.0722 - val_loss: 0.0628
Epoch 9/15
3072/2984 [==============================] - 8s - loss: 0.0719 - val_loss: 0.0602
Epoch 10/15
3072/2984 [==============================] - 8s - loss: 0.0668 - val_loss: 0.1097
Epoch 11/15
3072/2984 [==============================] - 8s - loss: 0.0791 - v